<a href="https://colab.research.google.com/github/NataHsH/GenerativeAI-Project/blob/NataliaHoncharova/LLM_finetuning_gpt2_wikitext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uninstall conflicting versions
!pip uninstall -y transformers
!pip uninstall -y torch

# Install the correct versions
!pip install transformers==4.29.0
!pip install torch==2.0.0





Found existing installation: transformers 4.29.0
Uninstalling transformers-4.29.0:
  Successfully uninstalled transformers-4.29.0
Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Successfully uninstalled torch-2.0.0
  Using cached transformers-4.29.0-py3-none-any.whl.metadata (111 kB)
Using cached transformers-4.29.0-py3-none-any.whl (7.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires torch>=1.11.0, which is not installed.
peft 0.15.2 requires torch>=1.13.0, which is not installed.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.29.0 which is incompatible.
  Using cached torch-2.0.0-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
Using cached torch-2.0.0-cp311-cp311-manylinux1_x86_64.whl (619.9 MB)
ERROR: pip's dependency resolver does not currently ta

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import wandb
wandb.login()  # login

wandb: Currently logged in as: honcharova-de (honcharova-de-hannover) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
from huggingface_hub import Repository
from huggingface_hub import upload_folder
from transformers import TrainerCallback





A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [6]:
import shutil
import os

# Define the cache directory path
cache_dir = "/root/.cache/huggingface/datasets"

# Check if the directory exists
if os.path.exists(cache_dir):
    # Recursively remove the directory and its contents
    shutil.rmtree(cache_dir)



In [7]:
# Initialize wandb for logging
wandb.init(
    project="GPT-2_Finetuning",
    name="finetuning_gpt2_wikitext",
    config={
        "epochs": 3,
        "batch_size": 4,
        "block_size": 128,
        "learning_rate": 5e-5,
        "vocab_size": 50257,  # Default vocab size for GPT-2
    }
)

# Define a callback to log losses to wandb
class WandbCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        # Log train and eval losses to wandb
        if logs.get("loss"):
            wandb.log({"train_loss": logs["loss"]})
        if logs.get("eval_loss"):
            wandb.log({"eval_loss": logs["eval_loss"]})

# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set pad token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Load dataset (Wikitext-103)
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

# Tokenize the data
def tokenize_function(examples):
    # Add labels equal to input_ids for language modeling
    encodings = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=wandb.config.block_size)
    encodings['labels'] = encodings['input_ids']
    return encodings

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Split dataset into train and test
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

# Define training parameters
training_args = TrainingArguments(
    output_dir="./gpt2_wikitext_finetuned",
    learning_rate=wandb.config.learning_rate,
    per_device_train_batch_size=wandb.config.batch_size,
    per_device_eval_batch_size=wandb.config.batch_size,
    num_train_epochs=wandb.config.epochs,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    report_to="wandb",
    evaluation_strategy="steps",
    eval_steps=500,
)

# Initialize the Trainer with the wandb callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    callbacks=[WandbCallback]
)

# Train the model
trainer.train()

# Save the trained model and tokenizer locally
model.save_pretrained("finetuned_gpt2_wikitext")
tokenizer.save_pretrained("finetuned_gpt2_wikitext")

# Log the final loss to wandb after training is complete
final_eval_loss = trainer.evaluate()['eval_loss']
wandb.log({"final_eval_loss": final_eval_loss})

# Upload the trained model and tokenizer to Hugging Face Hub
from huggingface_hub import login

# Log in with your Hugging Face token (replace with your actual token or handle this securely)
login(token="your_huggingface_token")  # Use your real token here

# Push the model and tokenizer to the Hub under a unique name
model.push_to_hub("gpt2_finetuned_wikitext")
tokenizer.push_to_hub("gpt2_finetuned_wikitext")

# Finish the wandb session
wandb.finish()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


KeyboardInterrupt: 